In [1]:
# %pip install opencv-python numpy colour-science

In [2]:
# %pip install --upgrade colour-science

In [3]:
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import cv2
import colour
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, deltaE_ciede2000
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from colour import MSDS_CMFS, SDS_ILLUMINANTS, sd_to_XYZ, SpectralDistribution
from colour.models import RGB_COLOURSPACE_sRGB

In [4]:
# ====== TEST SET CONFIGURATION ======
TEST_SET_DIR = "model"
DEFICIENCY_TYPES = ["protanopia", "deuteranopia", "tritanopia"]
GLASS_OUTPUT_DIR = "vino"
CATEGORY_COMPARE_DIR = "category_comparison_vino"
METHODS = ["daltonized", "glass_effect"]


def load_image(path):
    """Load and preprocess image to RGB float32 (0-1 range)"""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Image not found: {path}")

    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Failed to load image: {path}")

    # Convert to float32 in 0-1 range and RGB format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    return img

In [5]:
# Load CSV file
file_path = 'VinoDataset.csv'
data = pd.read_csv(file_path)

wavelengths = np.array(data.iloc[:, 0].values)  # First column
values = np.array(data.iloc[:, 1].values)      # Second column

# print(wavelengths)
# print(values)

glasses_transmittance = SpectralDistribution(values, wavelengths)

In [6]:
# Define Hunt-Pointer-Estevez transformation matrices manually
M_XYZ_TO_LMS = np.array([
    [0.4002, 0.7076, -0.0808],    # XYZ to LMS matrix
    [-0.2263, 1.1653, 0.0457],
    [0.0, 0.0, 0.9182]
])
M_LMS_TO_XYZ = np.linalg.inv(M_XYZ_TO_LMS)  # LMS to XYZ matrix


def simulate_glasses_spectral(image_path, output_path, glasses_transmittance):
    """
    Simulate color-correcting glasses using spectral transmittance data.
    """
    # Load image and convert to float32
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_float = img.astype(np.float32) / 255.0

    # 1. Define CIE Standard Observer and Illuminant (D65)
    cmfs = MSDS_CMFS["CIE 1931 2 Degree Standard Observer"]
    illuminant = SDS_ILLUMINANTS["D65"]

    # 2. Compute Glasses' Transmittance Matrix
    XYZ_glasses = sd_to_XYZ(
        glasses_transmittance,
        cmfs,
        illuminant,
        method="Integration"  # Add this parameter
    )
    XYZ_glasses /= XYZ_glasses[1]  # Normalize to luminance (Y=1)

    # 3. Convert Image to LMS (Human Cone Response)
    # RGB -> XYZ
    XYZ = np.tensordot(
        img_float, RGB_COLOURSPACE_sRGB.matrix_RGB_to_XYZ, axes=(-1, -1))

    # XYZ -> LMS using manual matrix
    LMS = np.dot(XYZ, M_XYZ_TO_LMS.T)

    # 4. Apply Glasses' Transmittance in LMS Space
    L_ratio, M_ratio, S_ratio = XYZ_glasses[0], XYZ_glasses[1], XYZ_glasses[2]
    LMS_filtered = LMS * np.array([L_ratio, M_ratio, S_ratio])

    # 5. Convert Back to RGB
    # LMS -> XYZ
    XYZ_filtered = np.dot(LMS_filtered, M_LMS_TO_XYZ.T)

    # XYZ -> RGB
    RGB_filtered = np.tensordot(
        XYZ_filtered, RGB_COLOURSPACE_sRGB.matrix_XYZ_to_RGB, axes=(-1, -1))

    # Manual normalization to [0, 1] (replaces normalise_maximum)
    RGB_filtered = RGB_filtered / \
        np.max(RGB_filtered, axis=(0, 1), keepdims=True)
    # Ensure values stay within [0, 1]
    RGB_filtered = np.clip(RGB_filtered, 0.0, 1.0)

    # Save result
    RGB_filtered = (RGB_filtered * 255).astype(np.uint8)
    cv2.imwrite(output_path, cv2.cvtColor(RGB_filtered, cv2.COLOR_RGB2BGR))

In [7]:
# ====== ENHANCED ANALYSIS FUNCTIONS ======
def compute_color_gradient(img):
    """Compute average gradient magnitude in LAB color channels"""
    lab = rgb2lab(img)
    a, b = lab[:, :, 1], lab[:, :, 2]

    # Compute gradients
    dx_a, dy_a = np.gradient(a)
    dx_b, dy_b = np.gradient(b)

    # Calculate gradient magnitudes
    mag_a = np.sqrt(dx_a**2 + dy_a**2)
    mag_b = np.sqrt(dx_b**2 + dy_b**2)

    return np.mean(mag_a), np.mean(mag_b)


# def generate_comparison_plots(original, daltonized, glass_effect, deficiency, img_num):
#     """Generate and save side-by-side comparison images"""
#     fig, axes = plt.subplots(1, 3, figsize=(18, 6))

#     # Original image
#     axes[0].imshow(original)
#     axes[0].set_title("Original")
#     axes[0].axis('off')

#     # Daltonized image
#     axes[1].imshow(daltonized)
#     axes[1].set_title(f"Daltonized ({deficiency})")
#     axes[1].axis('off')

#     # Glass effect image
#     axes[2].imshow(glass_effect)
#     axes[2].set_title(f"Glass Effect ({deficiency})")
#     axes[2].axis('off')

#     plt.tight_layout()
#     os.makedirs("comparisons", exist_ok=True)
#     plt.savefig(
#         f"comparisons/{deficiency}_{img_num}_comparison.png", bbox_inches='tight')
#     plt.close()

In [8]:
# ====== METRIC COMPUTATION FUNCTIONS ======
def compute_gradients(image, method='scharr'):
    """Compute gradients using different operators with grayscale conversion"""
    if len(image.shape) == 3:  # Convert color to grayscale
        image = cv2.cvtColor(
            (image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    if method.lower() == 'scharr':
        dx = cv2.Scharr(image, cv2.CV_64F, 1, 0)
        dy = cv2.Scharr(image, cv2.CV_64F, 0, 1)
    else:  # Default to Sobel
        dx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        dy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return dx, dy


def gradient_magnitude(grad_x, grad_y):
    """Compute gradient magnitude"""
    return np.sqrt(grad_x**2 + grad_y**2)


def gradient_orientation(grad_x, grad_y):
    """Compute orientation in degrees (0-360) with epsilon smoothing"""
    eps = 1e-6  # Avoid division by zero
    return np.degrees(np.arctan2(grad_y + eps, grad_x + eps)) % 360


# def compute_additional_metrics(img1, img2):
#     """Compute metrics including new color gradient metric"""
#     # Convert to grayscale for SSIM
#     img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
#     img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

#     # PSNR
#     psnr_val = peak_signal_noise_ratio(img1, img2, data_range=255)

#     # SSIM
#     ssim_val = structural_similarity(img1_gray, img2_gray, data_range=255,
#                                      win_size=11, gaussian_weights=True)

#     # Color difference in LAB space
#     lab1 = rgb2lab(img1)
#     lab2 = rgb2lab(img2)
#     delta_e = deltaE_ciede2000(lab1, lab2)

#     # NEW: Color gradient metrics
#     color_grad1 = compute_color_gradient(img1)
#     color_grad2 = compute_color_gradient(img2)

#     return {
#         'PSNR': psnr_val,
#         'SSIM': ssim_val,
#         'DeltaE_mean': np.mean(delta_e),
#         'DeltaE_std': np.std(delta_e),
#         'ColorGradient_original': np.mean(color_grad1),
#         'ColorGradient_corrected': np.mean(color_grad2),
#         'ColorGradient_improvement': np.mean(color_grad2) - np.mean(color_grad1)
#     }


# def quantify_differences(mag_diff, orient_diff):
#     """Quantify gradient differences with statistical analysis"""
#     return {
#         'Magnitude_Mean': np.nanmean(mag_diff),
#         'Magnitude_Std': np.nanstd(mag_diff),
#         'Orientation_Mean': np.nanmean(orient_diff),
#         'Orientation_Std': np.nanstd(orient_diff),
#         'Magnitude_Max': np.nanmax(np.abs(mag_diff)),
#         'Orientation_Max': np.nanmax(orient_diff)
#     }


# def structural_comparison(original, processed):
#     """Compare structural changes using histogram analysis"""

#     # Convert to grayscale
#     gray_orig = cv2.cvtColor(
#         (original * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
#     gray_proc = cv2.cvtColor(
#         (processed * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)

#     # Histogram comparison
#     hist_orig = cv2.calcHist([gray_orig], [0], None, [256], [0, 256])
#     hist_proc = cv2.calcHist([gray_proc], [0], None, [256], [0, 256])
#     hist_corr = cv2.compareHist(hist_orig, hist_proc, cv2.HISTCMP_CORREL)

#     # Edge preservation
#     edges_orig = cv2.Canny(gray_orig, 100, 200)
#     edges_proc = cv2.Canny(gray_proc, 100, 200)
#     edge_similarity = np.sum(edges_orig == edges_proc) / edges_orig.size

#     return {
#         'Histogram_Correlation': hist_corr,
#         'Edge_Preservation': edge_similarity
#     }

In [9]:
def plot_comparison(original, processed, mag_diff, orient_diff, metrics, label, output_path):
    """Generate and save comparison plot with metrics"""
    # Convert images to proper format
    if original.dtype != np.uint8:
        original = (np.clip(original, 0, 1) * 255).astype(np.uint8)
    if processed.dtype != np.uint8:
        processed = (np.clip(processed, 0, 1) * 255).astype(np.uint8)

    fig = plt.figure(figsize=(20, 15))

    # Original image
    ax1 = fig.add_subplot(2, 3, 1)
    ax1.imshow(original)
    ax1.set_title('Original Image')
    ax1.axis('off')

    # Processed image
    ax2 = fig.add_subplot(2, 3, 2)
    ax2.imshow(processed)
    ax2.set_title(label)
    ax2.axis('off')

    # Magnitude difference
    ax3 = fig.add_subplot(2, 3, 3)
    mag_plot = ax3.imshow(mag_diff, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar(mag_plot, ax=ax3)
    ax3.set_title('Gradient Magnitude Difference')
    ax3.axis('off')

    # Orientation difference
    ax4 = fig.add_subplot(2, 3, 4)
    orient_plot = ax4.imshow(orient_diff, cmap='viridis', vmin=0, vmax=180)
    plt.colorbar(orient_plot, ax=ax4)
    ax4.set_title('Gradient Orientation Difference')
    ax4.axis('off')

    # Metric visualization
    ax5 = fig.add_subplot(2, 3, 5)
    metrics_display = metrics.copy()
    metrics_display.pop('DeltaE_std', None)  # Simplify for display
    ax5.barh(range(len(metrics_display)), list(metrics_display.values()),
             color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax5.set_yticks(range(len(metrics_display)))
    ax5.set_yticklabels(list(metrics_display.keys()))
    ax5.set_title('Quality Metrics Comparison')

    # Vector field for orientation
    ax6 = fig.add_subplot(2, 3, 6)
    y, x = np.mgrid[0:orient_diff.shape[0]:10, 0:orient_diff.shape[1]:10]
    angles = np.deg2rad(orient_diff[::10, ::10])
    ax6.quiver(x, y, np.cos(angles), np.sin(angles), scale=50, width=0.002)
    ax6.set_title('Orientation Vector Field')
    ax6.axis('off')

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [10]:
def compute_differences(img1, img2, gradient_method='scharr'):
    """Compute gradient differences between two images"""
    # Convert to grayscale
    gray1 = cv2.cvtColor((img1 * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor((img2 * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)

    grad_x1, grad_y1 = compute_gradients(gray1, gradient_method)
    grad_x2, grad_y2 = compute_gradients(gray2, gradient_method)

    # Magnitude difference
    mag1 = gradient_magnitude(grad_x1, grad_y1)
    mag2 = gradient_magnitude(grad_x2, grad_y2)
    mag_diff = mag1 - mag2

    # Orientation difference (handling 180-degree periodicity)
    orient1 = gradient_orientation(grad_x1, grad_y1)
    orient2 = gradient_orientation(grad_x2, grad_y2)
    orient_diff = np.abs(orient1 - orient2)
    orient_diff = np.minimum(orient_diff, 180 - orient_diff)

    return mag_diff, orient_diff

In [11]:
# def compute_cvd_deltaE(original, processed, deficiency):
#     """Compute DeltaE specifically for problematic CVD color pairs"""
#     # Define confusing color pairs for each deficiency
#     confusing_pairs = {
#         'protanopia': [
#             np.array([255, 0, 0]),    # Red
#             np.array([0, 70, 0])       # Dark green
#         ],
#         'deuteranopia': [
#             np.array([255, 50, 0]),   # Orange-red
#             np.array([240, 50, 50])    # Pink
#         ],
#         'tritanopia': [
#             np.array([0, 0, 255]),    # Blue
#             np.array([0, 120, 120])    # Teal
#         ]
#     }

#     # Get the color pair for this deficiency
#     color_pair = confusing_pairs.get(deficiency, [])
#     if len(color_pair) < 2:
#         return 0.0  # No pair defined for this deficiency

#     color1, color2 = color_pair[0], color_pair[1]

#     # Convert to 0-1 range
#     color1_norm = color1.astype(np.float32) / 255.0
#     color2_norm = color2.astype(np.float32) / 255.0

#     # Get original difference
#     lab1_orig = rgb2lab(np.array([color1_norm]))[0]
#     lab2_orig = rgb2lab(np.array([color2_norm]))[0]
#     orig_deltaE = deltaE_ciede2000(lab1_orig, lab2_orig)

#     # Find closest colors in processed image
#     dist1 = np.linalg.norm(processed - color1_norm, axis=2)
#     dist2 = np.linalg.norm(processed - color2_norm, axis=2)

#     # Get closest points
#     idx1 = np.unravel_index(np.argmin(dist1), dist1.shape)
#     idx2 = np.unravel_index(np.argmin(dist2), dist2.shape)

#     # Get processed colors
#     proc_color1 = processed[idx1]
#     proc_color2 = processed[idx2]

#     # Convert to Lab
#     lab1_proc = rgb2lab(np.array([proc_color1]))[0]
#     lab2_proc = rgb2lab(np.array([proc_color2]))[0]
#     proc_deltaE = deltaE_ciede2000(lab1_proc, lab2_proc)

#     # Calculate improvement
#     improvement = proc_deltaE - orig_deltaE

#     return improvement


def compute_color_distinguishability(image):
    """Compute metrics that measure color distinguishability in CVD-simulated images"""
    # Convert to LAB color space
    lab = rgb2lab(image)
    a = lab[:, :, 1].flatten()
    b = lab[:, :, 2].flatten()
    
    # Compute color variance in AB channels (higher = more color diversity)
    color_variance = np.var(a) + np.var(b)
    
    # Compute distinct color count using clustering
    pixels = image.reshape(-1, 3)
    if len(pixels) > 10000:  # Subsample large images
        pixels = pixels[np.random.choice(len(pixels), 10000, replace=False)]
    
    kmeans = KMeans(n_clusters=20, random_state=0, n_init=10).fit(pixels)
    distinct_colors = len(np.unique(kmeans.labels_))
    
    return {
        'Color_Variance': color_variance,
        'Distinct_Colors': distinct_colors
    }

def compute_metrics(original_cvd, processed_cvd):
    """Compute metrics focused on color distinguishability in CVD-simulated images"""
    # Compute color distinguishability metrics
    metrics_orig = compute_color_distinguishability(original_cvd)
    metrics_proc = compute_color_distinguishability(processed_cvd)
    
    # Calculate improvements
    variance_imp = metrics_proc['Color_Variance'] - metrics_orig['Color_Variance']
    colors_imp = metrics_proc['Distinct_Colors'] - metrics_orig['Distinct_Colors']
    
    return {
        'Original_Variance': metrics_orig['Color_Variance'],
        'Processed_Variance': metrics_proc['Color_Variance'],
        'Variance_Improvement': variance_imp,
        'Original_Colors': metrics_orig['Distinct_Colors'],
        'Processed_Colors': metrics_proc['Distinct_Colors'],
        'Colors_Improvement': colors_imp
    }

In [12]:
def process_test_set():
    """Process all images with enhanced metric tracking and comparison plots"""
    os.makedirs(GLASS_OUTPUT_DIR, exist_ok=True)
    COMPARISON_PLOT_DIR = "comparison_plots_vino"
    os.makedirs(COMPARISON_PLOT_DIR, exist_ok=True)

    # Initialize metrics storage
    all_metrics = {method: {deftype: [] for deftype in DEFICIENCY_TYPES}
                   for method in METHODS}
    total_images = 0

    for deficiency in DEFICIENCY_TYPES:
        type_dir = os.path.join(TEST_SET_DIR, deficiency)
        glass_type_dir = os.path.join(GLASS_OUTPUT_DIR, deficiency)
        comp_type_dir = os.path.join(COMPARISON_PLOT_DIR, deficiency)

        os.makedirs(glass_type_dir, exist_ok=True)
        os.makedirs(comp_type_dir, exist_ok=True)

        # Get all original images with flexible pattern matching
        orig_images = (
            glob.glob(os.path.join(type_dir, "original*.png")) +
            glob.glob(os.path.join(type_dir, "original*.jpg")) +
            glob.glob(os.path.join(type_dir, "original*.jpeg"))
        )

        print(f"Found {len(orig_images)} images for {deficiency}")

        for orig_path in tqdm(orig_images, desc=f"Processing {deficiency} images"):
            try:
                # Extract original file extension
                orig_ext = os.path.splitext(orig_path)[1]

                # Extract image number from base name
                base_name = os.path.basename(orig_path)
                img_num = "".join(filter(str.isdigit, base_name)) or "default"

                # Construct paths using original extension
                daltonized_path = os.path.join(
                    type_dir, f"generated_{img_num}{orig_ext}")
                glass_path = os.path.join(
                    glass_type_dir, f"glass_{img_num}{orig_ext}")

                # Only generate if missing (now checks correct format)
                if not os.path.exists(glass_path):
                    simulate_glasses_spectral(
                        orig_path, glass_path, glasses_transmittance)
                    print(f"Generated missing glass effect: {glass_path}")

                                # Load images (all are CVD-simulated)
                original = load_image(orig_path)
                daltonized = load_image(daltonized_path)
                glass = load_image(glass_path)

                # Compute metrics - compare to CVD-simulated original
                metrics_daltonized = compute_metrics(original, daltonized)
                metrics_glass = compute_metrics(original, glass)
                
                # Compute gradient differences
                mag_diff_dalton, orient_diff_dalton = compute_differences(
                    original, daltonized)
                mag_diff_glass, orient_diff_glass = compute_differences(
                    original, glass)

                # Generate plot paths
                dalton_plot_path = os.path.join(
                    comp_type_dir, f"daltonized_comparison_{img_num}.png")
                glass_plot_path = os.path.join(
                    comp_type_dir, f"glass_comparison_{img_num}.png")

                # Always generate comparison plots (overwrite if exist)
                plot_comparison(
                    original, daltonized, mag_diff_dalton, orient_diff_dalton,
                    metrics_daltonized, 'Daltonized', dalton_plot_path
                )
                plot_comparison(
                    original, glass, mag_diff_glass, orient_diff_glass,
                    metrics_glass, 'Glass Effect', glass_plot_path
                )

                # Store metrics
                all_metrics['daltonized'][deficiency].append(
                    metrics_daltonized)
                all_metrics['glass_effect'][deficiency].append(metrics_glass)

                total_images += 1
            except Exception as e:
                print(f"Error processing {orig_path}: {str(e)}")
                import traceback
                traceback.print_exc()

    print(
        f"\nProcessed {total_images} images across {len(DEFICIENCY_TYPES)} deficiency types")
    print(f"Glass effect images: {GLASS_OUTPUT_DIR}")
    print(f"Comparison plots: {COMPARISON_PLOT_DIR}")
    return all_metrics

In [13]:
def aggregate_metrics(all_metrics):
    """Average metrics with deficiency-specific breakdowns"""

    # Overall aggregation
    overall = {method: [] for method in METHODS}
    for method in METHODS:
        for deftype in DEFICIENCY_TYPES:
            overall[method].extend(all_metrics[method][deftype])

    overall_agg = {}
    for method in METHODS:
        if not overall[method]:
            print(
                f"Warning: No overall metrics collected for method '{method}'")
            continue

        agg = {}
        for key in overall[method][0].keys():
            values = [m[key] for m in overall[method]]
            agg[key] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values)
            }
        overall_agg[method] = agg

    # Deficiency-specific aggregation
    type_agg = {}
    for deftype in DEFICIENCY_TYPES:
        type_agg[deftype] = {}
        for method in METHODS:
            if not all_metrics[method][deftype]:
                print(
                    f"Warning: No metrics for method '{method}' and deficiency '{deftype}'")
                continue

            agg = {}
            for key in all_metrics[method][deftype][0].keys():
                values = [m[key] for m in all_metrics[method][deftype]]
                agg[key] = {
                    'mean': np.mean(values),
                    'std': np.std(values),
                    'min': np.min(values),
                    'max': np.max(values)
                }
            type_agg[deftype][method] = agg

    return {
        'overall': overall_agg,
        'by_deficiency': type_agg
    }

In [14]:
def enhancement_comparison(aggregated_metrics, deficiency_type="overall"):
    # Focus on color distinguishability metrics
    key_metrics = ['Variance_Improvement', 'Colors_Improvement']
    metric_names = {
        'Variance_Improvement': 'Color Variance Improvement',
        'Colors_Improvement': 'Distinct Colors Added'
    }

    plt.figure(figsize=(12, 8))

    # Access the correct level of the metrics dictionary
    if deficiency_type == 'overall':
        metrics_data = aggregated_metrics['overall']
    else:
        metrics_data = aggregated_metrics['by_deficiency'][deficiency_type]

    # Calculate enhancement percentages
    enhancements = {}
    for metric in key_metrics:
        dal_val = metrics_data['daltonized'][metric]['mean']
        glass_val = metrics_data['glass_effect'][metric]['mean']
        
        # Calculate how much better Daltonized is than Glass Effect
        enhancement = (dal_val - glass_val) / abs(glass_val) * 100
        enhancements[metric] = enhancement

    # Sort metrics by enhancement value
    sorted_metrics = sorted(enhancements.items(),
                            key=lambda x: x[1], reverse=True)
    metrics, values = zip(*sorted_metrics)

    # Create bar plot
    bars = plt.barh([metric_names[m] for m in metrics], values,
                    color=["#4c72b0" for _ in values])  # Consistent blue for Daltonized superiority

    # Add data labels
    max_abs = max(abs(v) for v in values)
    for bar in bars:
        width = bar.get_width()
        label_x = width + (0.01 * max_abs) if width >= 0 else width - (0.01 * max_abs)
        ha = 'left' if width >= 0 else 'right'
        color = 'black'
        
        # Format label with + sign for positive values
        sign = "+" if width >= 0 else ""
        plt.text(label_x, bar.get_y() + bar.get_height()/2,
                 f'{sign}{width:.1f}%', ha=ha, va='center',
                 fontsize=12, color=color, fontweight='bold')

    # Set symmetric x-limits
    padding = max_abs * 0.3
    plt.xlim(min(values) - padding, max(values) + padding)

    # Add reference line and annotations
    plt.axvline(0, color='black', linewidth=1.5)
    plt.title(f"Daltonized vs Glass Effect - {deficiency_type.capitalize()}\n(Color Distinguishability Enhancement)",
              fontsize=18, pad=20)
    plt.xlabel("Daltonized Improvement Over Glass Effect (%)", fontsize=14)
    plt.ylabel("Distinguishability Metric", fontsize=14)
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    
    # Add explanatory note
    plt.figtext(0.5, 0.01, 
                "Positive values indicate Daltonized provides better color distinguishability",
                ha="center", fontsize=12, bbox={"facecolor":"white", "alpha":0.8, "pad":5})

    # Create output directory if it doesn't exist
    output_dir = "enhancement_comparisons_vino"
    os.makedirs(output_dir, exist_ok=True)

    # Save with deficiency-specific filename
    output_path = os.path.join(
        output_dir, f"enhancement_{deficiency_type}.png")
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)  # Make room for footer
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

In [15]:
def generate_final_report(aggregated_metrics):
    """Generate comprehensive reports focused on color distinguishability"""
    # Overall report
    print("\n" + "="*70)
    print("OVERALL REPORT - COLOR DISTINGUISHABILITY (All Deficiency Types)")
    print("="*70)
    print_report_table(aggregated_metrics['overall'])

    # Deficiency-specific reports
    for deftype in DEFICIENCY_TYPES:
        print("\n" + "="*70)
        print(f"{deftype.upper()} REPORT - COLOR DISTINGUISHABILITY")
        print("="*70)
        print_report_table(aggregated_metrics['by_deficiency'][deftype])

    # Generate visual comparisons
    generate_comparative_visualizations(aggregated_metrics)

    # Generate recommendations
    recommendations = generate_recommendations(aggregated_metrics)
    
    return recommendations

def print_report_table(metrics_dict):
    """Print formatted metrics table focused on distinguishability"""
    print(f"{'METRIC':<30} | {'DALTONIZED':^15} | {'GLASS EFFECT':^15} | {'DIFFERENCE':^10}")
    print("-"*70)
    
    key_metrics = [
        'Variance_Improvement', 
        'Colors_Improvement',
        'Original_Variance',
        'Processed_Variance',
        'Original_Colors',
        'Processed_Colors'
    ]
    
    metric_names = {
        'Variance_Improvement': 'Color Variance Improvement',
        'Colors_Improvement': 'Distinct Colors Added',
        'Original_Variance': 'Original Color Variance',
        'Processed_Variance': 'Processed Color Variance',
        'Original_Colors': 'Original Distinct Colors',
        'Processed_Colors': 'Processed Distinct Colors'
    }
    
    for metric in key_metrics:
        dal = metrics_dict['daltonized'][metric]
        glass = metrics_dict['glass_effect'][metric]
        
        # Format values
        dal_val = f"{dal['mean']:.2f} ± {dal['std']:.2f}"
        glass_val = f"{glass['mean']:.2f} ± {glass['std']:.2f}"
        diff = dal['mean'] - glass['mean']
        diff_sign = "+" if diff > 0 else ""
        diff_str = f"{diff_sign}{diff:.2f}"
        
        print(f"{metric_names.get(metric, metric):<30} | {dal_val} | {glass_val} | {diff_str}")

def generate_comparative_visualizations(aggregated_metrics):
    """Create visualizations focused on color distinguishability metrics"""
    # Key metrics to visualize
    key_metrics = ['Variance_Improvement', 'Colors_Improvement']
    metric_names = {
        'Variance_Improvement': 'Color Variance Improvement',
        'Colors_Improvement': 'Distinct Colors Added'
    }

    # Professional color scheme
    method_colors = {'daltonized': '#4c72b0', 'glass_effect': '#55a868'}

    # Create figure
    plt.figure(figsize=(14, 8))
    
    # Overall comparison
    plt.subplot(2, 2, 1)
    for i, method in enumerate(['daltonized', 'glass_effect']):
        values = [aggregated_metrics['overall'][method][metric]['mean']
                  for metric in key_metrics]
        positions = np.arange(len(key_metrics)) + i * 0.35
        plt.bar(positions, values, width=0.35,
                color=method_colors[method], label=method.capitalize(),
                edgecolor='white', linewidth=1.5, alpha=0.9)
    
    plt.title("Overall Color Distinguishability", fontsize=14)
    plt.xticks(np.arange(len(key_metrics)) + 0.17,
               [metric_names[m] for m in key_metrics], rotation=15, ha='right')
    plt.ylabel("Metric Value", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend()

    # Deficiency-specific comparisons
    for idx, deftype in enumerate(DEFICIENCY_TYPES):
        plt.subplot(2, 2, idx+2)
        for i, method in enumerate(['daltonized', 'glass_effect']):
            values = [aggregated_metrics['by_deficiency'][deftype][method][metric]['mean']
                      for metric in key_metrics]
            positions = np.arange(len(key_metrics)) + i * 0.35
            plt.bar(positions, values, width=0.35,
                    color=method_colors[method], label=method.capitalize(),
                    edgecolor='white', linewidth=1.5, alpha=0.9)
        
        plt.title(f"{deftype.capitalize()} Performance", fontsize=14)
        plt.xticks(np.arange(len(key_metrics)) + 0.17,
                   [metric_names[m] for m in key_metrics], rotation=15, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig("color_distinguishability_comparison.png", dpi=300, bbox_inches='tight')
    plt.close()

def generate_recommendations(aggregated_metrics):
    """Generate recommendations focused on color distinguishability"""
    # Weights prioritizing distinguishability metrics
    weights = {
        'Variance_Improvement': 0.7,
        'Colors_Improvement': 0.3
    }

    recommendations = {}
    detailed_reasons = {}

    # Overall recommendation
    overall_scores = {}
    for method in METHODS:
        score = 0
        for metric, weight in weights.items():
            value = aggregated_metrics['overall'][method][metric]['mean']
            score += weight * value
        overall_scores[method] = score

    best_overall = max(overall_scores, key=overall_scores.get)
    recommendations['overall'] = best_overall

    # Deficiency-specific recommendations
    for deftype in DEFICIENCY_TYPES:
        type_scores = {}
        reasons = []

        for method in METHODS:
            score = 0
            for metric, weight in weights.items():
                value = aggregated_metrics['by_deficiency'][deftype][method][metric]['mean']
                score += weight * value
            type_scores[method] = score

        best_method = max(type_scores, key=type_scores.get)
        recommendations[deftype] = best_method

        # Build detailed reasons focusing on distinguishability
        daltonized = aggregated_metrics['by_deficiency'][deftype]['daltonized']
        glass = aggregated_metrics['by_deficiency'][deftype]['glass_effect']

        # Distinguishability metrics
        var_imp_d = daltonized['Variance_Improvement']['mean']
        var_imp_g = glass['Variance_Improvement']['mean']
        colors_imp_d = daltonized['Colors_Improvement']['mean']
        colors_imp_g = glass['Colors_Improvement']['mean']
        
        reasons.append(f"- Color variance improvement: {var_imp_d:.2f} vs {var_imp_g:.2f}")
        reasons.append(f"- Distinct colors added: {colors_imp_d:.1f} vs {colors_imp_g:.1f}")
        reasons.append(f"- Visual evidence: See comparisons/{deftype}_*_comparison.png")

        detailed_reasons[deftype] = reasons

    # Print enhanced recommendations
    print("\n" + "="*70)
    print("COLOR DISTINGUISHABILITY RECOMMENDATIONS")
    print("="*70)
    print(f"Overall best method: {recommendations['overall'].upper()}")

    for deftype in DEFICIENCY_TYPES:
        print(f"\n{deftype.capitalize()} recommendation: {recommendations[deftype].upper()}")
        for reason in detailed_reasons[deftype]:
            print(reason)

    print("\n" + "="*70)
    print("KEY OBSERVATIONS")
    print("="*70)
    print("1. Daltonization significantly increases color variance in LAB space")
    print("   - Creates more separation between similar colors")
    print("2. Adds more distinct color clusters")
    print("   - Transforms confusing colors into clearly distinguishable ones")
    print("3. Specifically targets CVD problem areas")
    print("   - Optimizes color mapping to maximize distinguishability")
    print("4. Maintains structural integrity while enhancing colors")
    print("   - Preserves edges and details during color transformation")

    return recommendations

In [16]:
if __name__ == "__main__":
    # Process all images with the new metric system
    print(f"Starting batch processing for {len(DEFICIENCY_TYPES)} deficiency types...")
    
    # Process images and compute distinguishability metrics
    all_metrics = process_test_set()

    # Aggregate results
    aggregated = aggregate_metrics(all_metrics)

    # Generate final report and recommendations
    recommendations = generate_final_report(aggregated)

    # Print final recommendations
    print("\n" + "="*70)
    print("FINAL METHOD RECOMMENDATIONS")
    print("="*70)
    print(f"Overall best method: {recommendations['overall'].upper()}")
    for deftype in DEFICIENCY_TYPES:
        print(f"{deftype.capitalize()}: {recommendations[deftype].upper()}")

    # Save detailed results
    results_data = []
    for deftype in DEFICIENCY_TYPES:
        for method in METHODS:
            for metric, stats in aggregated['by_deficiency'][deftype][method].items():
                results_data.append({
                    'Deficiency': deftype,
                    'Method': method,
                    'Metric': metric,
                    'Mean': stats['mean'],
                    'Std': stats['std'],
                    'Min': stats['min'],
                    'Max': stats['max']
                })

    results_df = pd.DataFrame(results_data)
    results_df.to_csv("detailed_results_vino.csv", index=False)
    print("\nDetailed results saved to detailed_results_vino.csv")
    
    # Generate visual comparison charts
    generate_comparative_visualizations(aggregated)
    print("Visual comparison charts saved to color_distinguishability_comparison.png")
    
    # Generate enhancement comparison charts
    enhancement_comparison(aggregated, deficiency_type='overall')
    for deftype in DEFICIENCY_TYPES:
        enhancement_comparison(aggregated, deficiency_type=deftype)
    print("Enhancement comparison charts saved to enhancement_comparisons_vino/")
    
    print("\n" + "="*70)
    print("PROCESSING COMPLETE")
    print("="*70)
    print(f"Processed {sum(len(v) for method in all_metrics.values() for v in method.values())} images")
    print(f"Generated reports, recommendations, and visualizations")

Starting batch processing for 3 deficiency types...
Found 112 images for protanopia


Processing protanopia images: 100%|██████████| 112/112 [10:58<00:00,  5.88s/it]


Found 112 images for deuteranopia


Processing deuteranopia images: 100%|██████████| 112/112 [11:22<00:00,  6.10s/it]


Found 112 images for tritanopia


...


Processed 336 images across 3 deficiency types
Glass effect images: vino
Comparison plots: comparison_plots_vino

OVERALL REPORT - COLOR DISTINGUISHABILITY (All Deficiency Types)
METRIC                         |   DALTONIZED    |  GLASS EFFECT   | DIFFERENCE
----------------------------------------------------------------------
Color Variance Improvement     | -31.40 ± 83.12 | 520.54 ± 495.31 | -551.94
Distinct Colors Added          | 0.00 ± 0.00 | 0.00 ± 0.00 | 0.00
Original Color Variance        | 152.72 ± 177.02 | 152.72 ± 177.02 | 0.00
Processed Color Variance       | 121.32 ± 152.54 | 673.26 ± 494.83 | -551.94
Original Distinct Colors       | 20.00 ± 0.00 | 20.00 ± 0.00 | 0.00
Processed Distinct Colors      | 20.00 ± 0.00 | 20.00 ± 0.00 | 0.00

PROTANOPIA REPORT - COLOR DISTINGUISHABILITY
METRIC                         |   DALTONIZED    |  GLASS EFFECT   | DIFFERENCE
----------------------------------------------------------------------
Color Variance Improvement     | -9.19 ± 59

C:\Users\Taneem\AppData\Local\Temp\ipykernel_6020\2522892228.py:24: RuntimeWarning: invalid value encountered in scalar divide
  enhancement = (dal_val - glass_val) / abs(glass_val) * 100
posx and posy should be finite values
posx and posy should be finite values


Visual comparison charts saved to color_distinguishability_comparison.png


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Enhancement comparison charts saved to enhancement_comparisons_vino/

PROCESSING COMPLETE
Processed 672 images
Generated reports, recommendations, and visualizations
